# Project 2: Continuous Control

This project was a trigger to explore reinforcement learning frameworks. The framework which was used to solve the problem is [stable baselines 3](https://stable-baselines3.readthedocs.io/en/master/) which is implemented with pytorch. The problem was solved using A2C, PPO and TD3 on the 20 agent environment. Below one can find a description of each algorithm with references to the codebase, notes and graphs from the experiments, and finally a comparison of the effectiveness of the three algorithms on the problem at hand. 

## Usage of stable baselines 3

### About the library
The library has an interesting story. In an attempt to standardise implementations and give the ability to use recent reinforcement learning algorithms out of the box to benchmark problems, OpenAI created the library [baselines](https://github.com/openai/baselines). Though the project was sucessful, it still lacked consistency and ease of use. This led some users to fork the library and create a friendlier version with cleaner code, [stable baselines](https://stable-baselines.readthedocs.io/en/master). The main library is based on TensorFlow, but recently a new version of it was introduced, [stable baselines 3](https://stable-baselines3.readthedocs.io/en/master/), which uses PyTorch. The library has currently A2C, PPO and TD3 implemented and gives the ability to the user to set different hyper parameters and define the architectures of the policy and value networks. 

It must be stated that there are several other libraries which have similar structure. Ones which were considered were [reagent](https://github.com/facebookresearch/ReAgent), [dopamine](https://github.com/google/dopamine) and [kerasrl](https://keras-rl.readthedocs.io/en/latest/). Aside from those, one can find several implementations of more recent or specialized algorithms via [paperswithcode](https://paperswithcode.com/area/playing-games). 

### Wrapping the Unity environments
One of the steps needed in order to use the library was to wrap the Unity environments in an environment class compatible with it. One can find the wrappers' code in `reacher/unity_environment_wrappers.py`. Two wrappers were built:
- One for the single agent environment. This was straight forward as one only needed to translate the Unity environment to a OpenAI gym environment.
- One for the multi-agent environment. Unfortunately stable baselines support only multiple environments with single agents in this direction. Nevertheless, the one can define a customized environment inheriting from `VecEnv` and handle the multiple agent actions as if they come from different environments.

## Learning algorithms
Below is a description of A2C, PPO and TD3. I attempt below to connect the algorithms with their implementation in the codebase.

### Buffers
All models below use buffers to accumulate information from the environment i.e. tuples of the form `(observation, action, reward, done)`. TD3 and SAC use a relplay buffer, while A2C and PPO use a rollout buffer.

#### Replay buffer
This buffer contains an internal fixed-size array which is being filled with information and after reaching its capacity new datapoints overwrite the oldest ones. Sampling is done in a uniform way and no type of prioritized replay is supported yet. This buffer is used in TD3 and SAC and usually holds ~1e6 samples and currently can only be used in single agent mode. Its purpose is to decorrelate consecutive agent actions by suffling them. 

#### Rollout buffer
This one is a bit more complex. It maintains an internal queue, just like the replay buffer, but also contains logic in the method `compute_returns_and_advantage`. This method, faithful to its name, computes and keeps track of the returns and advantages. Those are estimates based on discounted sums of rewards coming from buffered games. They are later on used by the policy-based methods todefine a loss target for the policy estimating network. They come in two flavours:

- *discouted return with value bootstrapping*: 

    This is the simplest approach. Uses discounted rewards and the value estimations to approximate advantages:

    $R_t = \sum\limits_{l \in [0, t_{fin} - t]} \gamma^{l} r_{t + l}$,

    $\hat{A}_t = R_t - \hat{V}_t$

    where $t_{fin}$ is the last step of the episode and $\hat{V}_t$ the value estimate of the critique on step $t$. The above formulas are optically pleasing, but the code also has to deal with the edge cases of episode ends and rollouts stopping in the middle of episodes.

- *generalized advantage estimation (GAE)*: 

    This is a more general approach which attempts to find a sweet spot between the variance introduced by the noise in rewards and the bias of the value estimation. GAE approximates advantages using several estimators and then combines them using an additional discount. In order to get an idea here are the two extreme estimators one could use for $\hat{A}_t$:
    - $\hat{A}_t =_{def} \delta_t = r_t + \gamma \hat{V}(s_t + 1) − \hat{V}(s_t)$
    
        This one has high bias as it heavily relies on the belief of $\hat{V}$ on what difference of values around step $t$ are.
    - $\hat{A}_t=_{def} \sum\limits_{l \in [0, t_{fin} - t]} \gamma^{l} \delta_{t + l} = \sum\limits_{l \in [0, t_{fin} - t]} \gamma^{l} r_{t + l} - \hat{V}(s_t)$
    
        This one has low bias as it is using $\hat{V}$ only to get an approximation of the value near step $t$ and then compares it with the discouted sum of rewards to figure out the difference. Nevertheless, it has high variance since the rewards change a lot, especially in the beginning of training.
        
    GAE considers the following spectrum between those two estimators:
    
    $\hat{A}_t^{(k)} =_{def} \sum\limits_{l \in [0, k - 1]} \gamma^{l} \delta_{t + l}$
    
    and finally combines them to:
    
    $\hat{A}^{GAE(\gamma, \lambda)}_t =_{def} (1 - \lambda) \sum\limits_{k \in [0, t_{fin} - t]} \lambda^{k} \hat{A}_t^{(k + 1)}$
    
    Note that the above, extreme, versions correspond to $GAE(\gamma, 0)$ and $GAE(\gamma, 1)$. For more details, check [the relevant publication](https://arxiv.org/abs/1506.02438).
    

**Note:** This buffer supports multiple environments, which in our case translates to multiple agents. The number of environments/agents changes the batch size during training to `batch_size * number_of_agents`. This is a great chance to increase the learning rate and speed up training, but one also has to keep in mind that a change to the number of agents leads to an indirect change of the hyper parameter setup.

### Generic algorithm structure
All algorithms in the library follow the same general steps:

```
while not exceeding the total number of steps:
    collect rollouts, i.e. run multiple times the current agents in the environment and gather data with a buffer
    
    loop over the data collected from the buffer - either sample, loop once over all, or loop for several epochs:
        update the actor and critique network approximators using gradient descent
```

What makes each one different, is the way of iterating over the buffered data and the choice of the loss function based on the actor and critique approximators.

### A2C


### PPO

### TD3

## Experiments and solution of the enviroment

### PPO trained on the single agent environment
- Setting the network architecture
- Picking the right learning rate
- Results

### PPO trained on the multi-agent environment
- Big speed-up
- More robust on learning rate changes
- Results

### A2C
- Trained also on  the multi-agent environment
- Sometimes converges really fast
- Not as consistent in the results. Some times converges, some not.
- Results

### TD3
-
-
-

## Next steps

### Learning faster
The solutions to the problem are extremely close to the optimal score, 40, so there is not a lot to improve regarding average rewards. On the other hand one could explore options to make the convergence of training faster. This could either mean needing fewer samples from the environment (thus more applicable in learning with real robots) or less training time from the user's perspective. Here are some ideas in this direction:
- *Train using SAC with tuned hyperparameters from the zoo*: This was proposed by [Antonin Raffin](https://github.com/araffin) in this [issue](https://github.com/koulakis/reacher-deep-reinforcement-learning/issues/3) opened in the repo. It interesting in general to use tunen hyper parameters from the [baselines 3 zoo](https://github.com/DLR-RM/rl-baselines3-zoo) for the HalfCheetah environment which is quite close to reacher and apply it to all algorithms used here.
- *Tune the size of the rollout and the number of epochs.* Up to now the default values of 2048 environment steps till rollout and 10 epochs for training. One could do an ablation study on both parameters and check how the reduction of the 'collect data -> train' cycle affects the speed of convergence.
- Mixed precision training: This a generic low-level improvement. In a lot of use cases, especially in computer vision, one can sucessfully reduce the float precision to 16 bit on specific parts of the networks trained and achiece speed increase in both training and predicting with minimal drops in performance. Curious if this works equally well in the reinforcement learning setting.

### Make the movements of the robot joint smoother
One can notice from the giff in the readme that some robot arms are shaking a bit while reaching the target. This is allowed in the sense that it doesn't prevent an agent in the simulated environment from reaching its goal. Nevertheless it is a bit far away from real-life where robots have limits in the speed and frequency of movement and also wear of faster through such behaviour. Here are some steps of improving towards this direction:

- *Use a continuity penalty*: This was also proposed by [Antonin Raffin](https://github.com/araffin). One would need to make some changes in the baselines codebase to achieve this. The rollout buffer is a good place to implement that, given that the estimates of advantage are already based on consecutive steps of the agent (which is needed in order to measure the speed of changes and aggregate it to a penalty added to the loss).
- *Use [gSDE](https://arxiv.org/abs/2005.05719)*: This is a very recent advance in the direction of learning with non-simulated agents. The idea to use it was, once more, suggested by [Antonin Raffin](https://github.com/araffin), who is actually one of two scientists who introduced the method. The method aims more in reducing the shaking behaviour throughout the whole training process, but may have an impact in the final behaviour of the agents, plus it is very cheap to try out given it is implemented in the baselines. It is also interesting to generate giffs throughout training and check how it affects the learning process.